In [1]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import gc
import time
gc.enable()
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import re
import requests
import folium
import branca.colormap as cm
import geopy
from tqdm import tqdm_notebook as tqdm
import json
import os
import geojson

warnings.filterwarnings('ignore')

In [4]:
%%time
# Load from preprocessed data
sd = pd.read_feather('../data/Final_Divvy_data_2018.feather')

CPU times: user 78.8 ms, sys: 234 ms, total: 313 ms
Wall time: 296 ms


In [6]:
sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216478 entries, 0 to 216477
Data columns (total 58 columns):
station_id                     216478 non-null int64
month                          216478 non-null int64
day                            216478 non-null int64
total_in                       216478 non-null float64
total_out                      216478 non-null float64
year                           216478 non-null float64
lon_ave                        216478 non-null float64
lat_ave                        216478 non-null float64
dp_max                         216478 non-null float64
dp_min                         216478 non-null float64
city_Chicago                   216478 non-null float64
city_Evanston                  216478 non-null float64
city_Oak_Park                  216478 non-null float64
days_online                    216478 non-null int64
dayofweek                      216478 non-null int64
apparentTemperatureHigh        216478 non-null float64
apparentTemperature

In [16]:
def get_data_of_a_day(month, day):
    sub_df = sd[(sd['month'] == month) & (sd['day'] == day)]
    lat_arr = np.array(sub_df.lat_ave)
    lon_arr = np.array(sub_df.lon_ave)
    flg_arr = np.array(sub_df.total_in > sub_df.dp_max)
    return lat_arr, lon_arr, flg_arr

In [58]:
map_from = folium.Map(location = [41.90, -87.64], zoom_start = 12,tiles=None)
folium.TileLayer(tiles='cartodbpositron', overlay=True, max_zoom=13, min_zoom=10.5).add_to(map_from)
d1_group = folium.FeatureGroup(name="July 1st, 2018 (Sun)", overlay=False)
d2_group = folium.FeatureGroup(name="July 2nd, 2018 (Mon)", overlay=False)
d3_group = folium.FeatureGroup(name="Oct 28th, 2018 (Sun)", overlay=False)
d4_group = folium.FeatureGroup(name="Oct 29th, 2018 (Mon)", overlay=False)


lat_lst_d1, long_lst_d1, flag_d1 = get_data_of_a_day(7, 1)
lat_lst_d2, long_lst_d2, flag_d2 = get_data_of_a_day(7, 2)
lat_lst_d3, long_lst_d3, flag_d3 = get_data_of_a_day(10, 28)
lat_lst_d4, long_lst_d4, flag_d4 = get_data_of_a_day(10, 29)

rd = 8
for lat, long, f in zip(lat_lst_d1, long_lst_d1, flag_d1):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d1_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d1_group)
    
for lat, long, f in zip(lat_lst_d2, long_lst_d2, flag_d2):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d2_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d2_group)    

for lat, long, f in zip(lat_lst_d3, long_lst_d3, flag_d3):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d3_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d3_group) 

for lat, long, f in zip(lat_lst_d4, long_lst_d4, flag_d4):
    if f:
        folium.CircleMarker([lat, long], color="red", fill=True, radius=rd, weight=0, fill_opacity=0.5).add_to(d4_group)
    else:
        folium.CircleMarker([lat, long], color="orange", fill=True, radius=rd, weight=0, fill_opacity=0.3).add_to(d4_group) 


d1_group.add_to(map_from)
# d2_group.add_to(map_from)
# d3_group.add_to(map_from)
d4_group.add_to(map_from)


folium.LayerControl(collapsed=False).add_to(map_from)
map_from.save('example_usage.html')

In [61]:
emtpy_m = folium.Map(location = [41.90, -87.64], zoom_start = 12,tiles=None)
folium.TileLayer(tiles='Stamen Watercolor', overlay=True).add_to(emtpy_m)

emtpy_m.save('empty_map.html')